In [1]:
1

1

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
data = {
    "number of rooms": np.random.randint(2, 8, 100),
    "house area": np.random.uniform(50, 200, 100),
    "yard area": np.random.uniform(20, 100, 100),
    "house price": lambda x: (
        50000 + 10000 * x["number of rooms"]
        + 300 * x["house area"]
        + 100 * x["yard area"]
        + np.random.normal(0, 10000, size=len(x["number of rooms"]))
    )
}

In [5]:
data["house price"] = data["house price"](data)

df = pd.DataFrame(data)
df.to_csv("house_prices.csv", index=False)

In [6]:
df.head()

,number of rooms,house area,yard area,house price
0,4,124.614718,24.518006,139653.636903
1,6,55.233640,53.104084,140856.605516
2,6,138.119440,49.359160,141484.103554
3,5,167.920705,48.279888,154248.970663
4,4,89.849826,41.131608,116724.139568


In [7]:
df = pd.read_csv("/root/sammyon7/Dummy_House.csv")
X = df[["number of rooms", "house area", "yard area"]].values
y = df["house price"].values.reshape(-1, 1)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)
y_train = scaler_y.fit_transform(y_train)
y_test = scaler_y.transform(y_test)

In [10]:
# Convert to PyTorch tensors!
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

In [ ]:
X_train

# HERE WE COMPARE THE HYPERDNN AND THE CASUAL DNN

In [12]:
class DNN(nn.Module):
    def __init__(self, input_dim):
        super(DNN, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.layers(x)

In [13]:
class HyperDNN(nn.Module):
    def __init__(self, input_dim, context_dim=3):
        super(HyperDNN, self).__init__()
        self.hypernetwork = nn.Sequential(
            nn.Linear(context_dim, 64),
            nn.ReLU(),
            nn.Linear(64, input_dim * 64)
        )
        self.dnn = nn.Sequential(
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x, context):
        # Generate weights for the first layer
        hyper_weights = self.hypernetwork(context).view(-1, x.size(1), 64)
        x = torch.bmm(x.unsqueeze(1), hyper_weights).squeeze(1)
        return self.dnn(x)

In [14]:
def train(model, optimizer, criterion, X, y, epochs=100, batch_size=16, is_hyper=False):
    for epoch in range(epochs):
        model.train()
        for i in range(0, len(X), batch_size):
            X_batch = X[i:i+batch_size]
            y_batch = y[i:i+batch_size]

            optimizer.zero_grad()

            if is_hyper:
                context = torch.ones(X_batch.size(0), 3)  # Dummy context
                preds = model(X_batch, context)
            else:
                preds = model(X_batch)

            loss = criterion(preds, y_batch)
            loss.backward()
            optimizer.step()

        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

In [15]:
dnn_model = DNN(input_dim=3)
dnn_optimizer = optim.Adam(dnn_model.parameters(), lr=0.01)
criterion = nn.MSELoss()

In [21]:
print("Training DNN...")
train(dnn_model, dnn_optimizer, criterion, X_train, y_train, epochs=5000)

Training DNN...
Epoch 10/5000, Loss: 0.0127
Epoch 20/5000, Loss: 0.0258
Epoch 30/5000, Loss: 0.0133
Epoch 40/5000, Loss: 0.0198
Epoch 50/5000, Loss: 0.0124
Epoch 60/5000, Loss: 0.0104
Epoch 70/5000, Loss: 0.0157
Epoch 80/5000, Loss: 0.0284
Epoch 90/5000, Loss: 0.0071
Epoch 100/5000, Loss: 0.0183
Epoch 110/5000, Loss: 0.0082
Epoch 120/5000, Loss: 0.0052
Epoch 130/5000, Loss: 0.0145
Epoch 140/5000, Loss: 0.0139
Epoch 150/5000, Loss: 0.0194
Epoch 160/5000, Loss: 0.0173
Epoch 170/5000, Loss: 0.0088
Epoch 180/5000, Loss: 0.0073
Epoch 190/5000, Loss: 0.0078
Epoch 200/5000, Loss: 0.0166
Epoch 210/5000, Loss: 0.0295
Epoch 220/5000, Loss: 0.0157
Epoch 230/5000, Loss: 0.0031
Epoch 240/5000, Loss: 0.0097
Epoch 250/5000, Loss: 0.0078
Epoch 260/5000, Loss: 0.0051
Epoch 270/5000, Loss: 0.0055
Epoch 280/5000, Loss: 0.0112
Epoch 290/5000, Loss: 0.0050
Epoch 300/5000, Loss: 0.0028
Epoch 310/5000, Loss: 0.0053
Epoch 320/5000, Loss: 0.0124
Epoch 330/5000, Loss: 0.0290
Epoch 340/5000, Loss: 0.0117
Epoch 3

In [22]:
hyperdnn_model = HyperDNN(input_dim=3)
hyperdnn_optimizer = optim.Adam(hyperdnn_model.parameters(), lr=0.01)

print("\nTraining HyperDNN...")
train(hyperdnn_model, hyperdnn_optimizer, criterion, X_train, y_train, epochs=5000, is_hyper=True)


Training HyperDNN...
Epoch 10/5000, Loss: 0.2161
Epoch 20/5000, Loss: 0.2394
Epoch 30/5000, Loss: 0.2060
Epoch 40/5000, Loss: 0.1918
Epoch 50/5000, Loss: 0.2119
Epoch 60/5000, Loss: 0.1482
Epoch 70/5000, Loss: 0.1359
Epoch 80/5000, Loss: 0.1328
Epoch 90/5000, Loss: 0.1269
Epoch 100/5000, Loss: 0.1159
Epoch 110/5000, Loss: 0.1251
Epoch 120/5000, Loss: 0.0903
Epoch 130/5000, Loss: 0.1052
Epoch 140/5000, Loss: 0.0866
Epoch 150/5000, Loss: 0.0806
Epoch 160/5000, Loss: 0.0783
Epoch 170/5000, Loss: 0.1067
Epoch 180/5000, Loss: 0.0570
Epoch 190/5000, Loss: 0.0635
Epoch 200/5000, Loss: 0.0520
Epoch 210/5000, Loss: 0.0629
Epoch 220/5000, Loss: 0.0473
Epoch 230/5000, Loss: 0.0418
Epoch 240/5000, Loss: 0.0408
Epoch 250/5000, Loss: 0.0469
Epoch 260/5000, Loss: 0.0306
Epoch 270/5000, Loss: 0.0390
Epoch 280/5000, Loss: 0.0507
Epoch 290/5000, Loss: 0.0488
Epoch 300/5000, Loss: 0.0300
Epoch 310/5000, Loss: 0.0330
Epoch 320/5000, Loss: 0.0506
Epoch 330/5000, Loss: 0.0317
Epoch 340/5000, Loss: 0.0270
E

In [23]:
def evaluate(model, X, y, is_hyper=False):
    model.eval()
    with torch.no_grad():
        if is_hyper:
            context = torch.ones(X.size(0), 3)
            preds = model(X, context)
        else:
            preds = model(X)
        loss = criterion(preds, y)
        print(f"Evaluation Loss: {loss.item():.4f}")

print("\nEvaluating DNN...")
evaluate(dnn_model, X_test, y_test)

print("\nEvaluating HyperDNN...")
evaluate(hyperdnn_model, X_test, y_test, is_hyper=True)


Evaluating DNN...
Evaluation Loss: 0.4061

Evaluating HyperDNN...
Evaluation Loss: 0.3011
